# ETL-Prozess: Standardlastprofil
---
Diese Komponente ist für das Extrahieren, Transformieren und Laden von Daten aus der Quelle [Stromnetz Hamburg](https://www.stromnetz-hamburg.de/fuer-partner/stromlieferanten/zaehlverfahren) zuständig. Die Komponente verarbeitet eine Quelldatei, die ein [synthetisches Standardlastprofil](https://www.enargus.de/pub/bscw.cgi/d12072-2/*/*/Standardlastprofil.html?op=Wiki.getwiki#:~:text=Das%20synthetische%20Lastprofilverfahren%20stellt%20eine,gewichtet%20und%20anschließend%20aufsummiert%20werden.) darstellt.

# Funktionsweise des Prozesses
---
In diesem Prozess werden die folgenden Funktionen verwendet:

1. `extraction_and_processing_dataset_Stromnetz_Hamburg()`: Die Funktion extrahiert die Daten aus einer [Quelldatei](https://filehub.admiralcloud.com/v5/deliverFile/76eee0f0-6112-4ca5-8f39-96d53f5871b8?download=true) und berechnet die Basisprozentwerte, die als Referenzwerte in den Simulationen zu verwenden sind.


2. `run_ETL_Standardlastprofil()`: Funktion zur Ausführung des Prozesses.



# Quellcode der Funktionen
---
Hier befindet sich der Code für die Funktionen, die im Abschnitt "Funktionsweise des Prozesses" beschrieben sind.

In [4]:
import openpyxl
import os
from openpyxl.styles import Font
from datetime import datetime, timedelta

output_folder = "Output-Data"

def extraction_and_processing_dataset_Stromnetz_Hamburg():
    try:
        # Definition der Pfade für die Eingabe- und Ausgabedateien
        input_folder = "Datenquelle/Stromnetz Hamburg-Daten"
        input_file = "Standardlastprofil Haushalt 2023.xlsx"
        input_path = os.path.join(input_folder, input_file)
        output_file = "ETL-Prozess_Ergebnisse.xlsx"
        output_path = os.path.join(output_folder, output_file)
        
        # Öffnen der Excel-Datei
        workbook = openpyxl.load_workbook(input_path)
        
        # Öffnen des gewünschten Arbeitsblatts
        sheet = workbook["Stromnetz Hamburg"]
        
        # Berechnung des gesamten Verbrauchs in kWh (Spalte C)
        gesamtverbrauch_kWh = sum(sheet[f"C{row}"].value for row in range(2, sheet.max_row + 1) if sheet[f"C{row}"].value is not None)
        
        # Hinzufügen einer neuen Spalte für den prozentualen Basiswert
        sheet["D1"].value = "SLP-Basisprozentwert"
        
        # Berechnung des prozentualen Werts und Eintrag in die neue Spalte
        for row in range(2, sheet.max_row + 1):
            verbrauch = sheet[f"C{row}"].value
            if verbrauch is not None:
                prozentwert = (verbrauch / gesamtverbrauch_kWh) * 100
                sheet[f"D{row}"].value = prozentwert
        
        ###########################################################################################################
        # Generierung der Datumswerte in 15-Minuten-Intervallen und Speichern in der Spalte "Datum"
        ###########################################################################################################
        
        # Definieren des Start- und Enddatums
        start_datum = datetime.strptime('2023-01-01 00:15', '%Y-%m-%d %H:%M')
        end_datum = datetime.strptime('2024-01-01 00:00', '%Y-%m-%d %H:%M')
        
        # Liste für die Speicherung der Datumswerte in 15-Minuten-Intervallen
        datumswerte = []
        aktuelles_datum = start_datum
        
        # Generierung der Datumswerte in 15-Minuten-Intervallen
        while aktuelles_datum <= end_datum:
            datumswerte.append(aktuelles_datum)
            aktuelles_datum += timedelta(minutes=15)
        
        # Einfügen der Datumswerte in die Spalte "Datum"
        for idx, datum in enumerate(datumswerte, start=2):  # Beginnt ab Zeile 2
            sheet[f"A{idx}"].value = datum
        
        ###########################################################################################################
        # Weitere Formatierungen und Speichern der Excel-Datei
        ###########################################################################################################
        
        # Entfernen der Fettschrift in den Zellen
        sheet["A1"].font = Font(bold=False)
        sheet["B1"].font = Font(bold=False)
        sheet["C1"].font = Font(bold=False)
        
        # Umbenennen des Arbeitsblatts
        sheet.title = "Simulationsparameter"
        
        # Speichern der Änderungen im Ausgabe-Excel-Datei
        workbook.save(output_path)
        
        print("- ETL-Prozess_Standardlastprofil: Aufgabe 1/1 erledigt")
    
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

In [5]:
def run_ETL_Standardlastprofil():
    extraction_and_processing_dataset_Stromnetz_Hamburg()